In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
import matplotlib.pyplot as plt

In [23]:
# Load the dataset
df = pd.read_csv('https://raw.githubusercontent.com/mwitiderrick/stockprice/master/NSE-TATAGLOBAL.csv')

# Filter and select the 'Close' column
data = df.filter(['Close'])

# Convert the data to a numpy array
dataset = data.values

# Get the number of rows to train the model on
training_data_len = int(np.ceil(0.8 * len(dataset)))

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

# Create the training data set
train_data = scaled_data[0:training_data_len, :]

# Split the data into x_train and y_train data sets
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])

# Convert x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape the data to 3D
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [24]:
# Initialize the model
model = Sequential()

# Add the first LSTM layer and some Dropout regularization
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))

# Add the second LSTM layer and some Dropout regularization
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

# Add the third LSTM layer and some Dropout regularization
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))

# Add the output layer
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')


In [25]:
model.fit(x_train, y_train, batch_size=32, epochs=100)


Epoch 1/100
49/49 [==============================] - 19s 102ms/step - loss: 0.0120
Epoch 2/100
49/49 [==============================] - 5s 99ms/step - loss: 0.0033
Epoch 3/100
49/49 [==============================] - 4s 75ms/step - loss: 0.0031
Epoch 4/100
49/49 [==============================] - 4s 75ms/step - loss: 0.0028
Epoch 5/100
49/49 [==============================] - 4s 92ms/step - loss: 0.0025
Epoch 6/100
49/49 [==============================] - 4s 86ms/step - loss: 0.0024
Epoch 7/100
49/49 [==============================] - 4s 75ms/step - loss: 0.0023
Epoch 8/100
49/49 [==============================] - 4s 88ms/step - loss: 0.0021
Epoch 9/100
49/49 [==============================] - 4s 83ms/step - loss: 0.0020
Epoch 10/100
49/49 [==============================] - 4s 76ms/step - loss: 0.0019
Epoch 11/100
49/49 [==============================] - 4s 80ms/step - loss: 0.0018
Epoch 12/100
49/49 [==============================] - 4s 90ms/step - loss: 0.0021
Epoch 13/100
49/49 [===

In [26]:
# Create the testing data set
test_data = scaled_data[training_data_len - 60:, :]

# Split the data into x_test and y_test data sets
x_test = []
y_test = dataset[training_data_len:, :]

for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])

# Convert x_test to a numpy array
x_test = np.array(x_test)

# Reshape the data to 3D
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# Get the model's predicted values
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

# Get the root mean squared error (RMSE)
rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
print('RMSE:', rmse)

# Plot the data
train = data[:training_data_len]
valid = data[training_data_len:]
valid


13/13 [==============================] - 2s 28ms/step
RMSE: 3.4860989146163486


,Close
1628,120.40
1629,120.55
1630,123.35
1631,125.55
1632,120.70
...,...
2030,118.65
2031,117.60
2032,120.65
2033,120.90
